In [1]:
import pandas as pd
from PIL import Image
import os
import numpy as np
from sklearn.metrics import confusion_matrix, accuracy_score, recall_score, f1_score
# from concurrent.futures import ThreadPoolExecutor, as_completed

In [2]:
# first, preprocess data

df = pd.read_csv("./metadata/training_mixed_frames_labels.csv", header=0)

print(df.shape[0])

print(df.iloc[0])

class0Data = df[(df["label"] == 0) | (df["label"] == 7)]

class1Data = df[(df["label"] == 1) | (df["label"] == 2) | (df["label"] == 3) | (df["label"] == 4) | (df["label"] == 17)]

class0Data = class0Data.iloc[::2]
class1Data = class1Data.iloc[::2]

class0Data["label"] = 0
class0Data = class0Data.sample(class1Data.shape[0], random_state=42)

class1Data["label"] = 1

print(class0Data.shape[0])
print(class1Data.shape[0])

finalDf = pd.concat([class0Data, class1Data], ignore_index=True)


# exclude_indexes = class0Data.index.union(class1Data.index)
# df_filtered = df.drop(exclude_indexes)
# print(df_filtered.shape)


# for frame in range(df_filtered.shape[0]):
#     currentImageIndex = df_filtered.iloc[frame]["frame_id"]
#     imagePath = f"./unlabeled_set/{currentImageIndex:06d}_512_512.jpg"
#     if os.path.exists(imagePath):
#         os.remove(imagePath)
#         print(f"{imagePath} deleted.")

# # list all items in the folder
# children = os.listdir("./unlabeled_set")

# print(f"Number of items in: {len(children)}")


144603
frame_id    0
label       1
Name: 0, dtype: int64
10796
10796


In [3]:
images = np.zeros((finalDf.shape[0], 96, 96), dtype=np.float32)
print(images.shape)  # (100, 512, 512)

def returnProcessedImage(dfIndex, pictureNumber):
    currentImage = Image.open(f"dataset/{pictureNumber:06d}_512_512.jpg")
    # convert this to 96 x 96 and grayscale
    processedImage = currentImage.resize([96, 96], Image.Resampling.LANCZOS).convert("L")
    normalizedNpArrImage = np.array(processedImage, dtype=np.float32) / 255.0

    return (dfIndex, normalizedNpArrImage)

# you guys ever just spend a good amount of time  on something only to realize it's all useless? yeah that's what happened right below

# currentDfIndex = 0
# batchSize = 10
# while(currentDfIndex < finalDf.shape[0]):
#     with ThreadPoolExecutor(max_workers=10) as executor:
#             futures = []
#             for i in range(0, min(finalDf.shape[0], currentDfIndex + batchSize), 1):
#                  dfIndex = currentDfIndex + i
#                  futures.append(executor.submit(returnProcessedImage, dfIndex, finalDf.iloc[dfIndex]["frame_id"]))
#             for future in as_completed(futures):
#                  result = future.result()
#                  images[result[0]] = result[1]
#     currentDfIndex += batchSize
#     if(currentDfIndex % 2000 == 0):
#         print(f"Made it to index {currentDfIndex}!")

for i in range(0, finalDf.shape[0], 1):
    actualPicIndex = finalDf.iloc[i]["frame_id"]
    currentImage = Image.open(f"dataset/{actualPicIndex:06d}_512_512.jpg")
    # convert this to 96 x 96 and grayscale
    processedImage = currentImage.resize([96, 96], Image.Resampling.LANCZOS).convert("L")
    normalizedNpArrImage = np.array(processedImage, dtype=np.float32) / 255.0

    images[i] = normalizedNpArrImage

    if(i % 2000 == 0):
        print(f"Made it to index {i}!")

(21592, 96, 96)
Made it to index 0!
Made it to index 2000!
Made it to index 4000!
Made it to index 6000!
Made it to index 8000!
Made it to index 10000!
Made it to index 12000!
Made it to index 14000!
Made it to index 16000!
Made it to index 18000!
Made it to index 20000!


In [4]:
labels = finalDf["label"].to_numpy()

# now shuffle them when passing in
randomIndexOrder = np.random.permutation(labels.shape[0])

images = images[randomIndexOrder]
labels = labels[randomIndexOrder]

# also must add an extra dimension
if images.ndim == 3:  # shape = (num_samples, 96, 96)
    images = images[..., np.newaxis]  # add channel dim
elif images.ndim != 4:
    images = np.squeeze(images, axis=-1)

# now make a training and test set
testPercentage = 0.85
cutoffPoint = int(testPercentage * labels.shape[0])

trainingXSet = images[0:cutoffPoint]
trainingYSet = labels[0:cutoffPoint]

testingXSet = images[cutoffPoint:]
testingYSet = labels[cutoffPoint:]



In [5]:
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Sequential
import tensorflow as tf

In [6]:
model = Sequential([
    # 96 x 96
    Conv2D(16, (3, 3), activation='relu', padding="same", input_shape=(96, 96, 1)),
    # becomes 48 x 48 after
    MaxPooling2D((2, 2)),

    # 
    Conv2D(32, (2, 2), padding="same", activation='relu'),
    GlobalAveragePooling2D(),

    Dense(32, activation='relu'),
    Dense(1, activation='sigmoid')  # for binary classification
])

# tensorflow is werid, so apparently you gotta compile it first? interesting
model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy', 'AUC', 'Precision', 'Recall']
)

history = model.fit(
    trainingXSet, trainingYSet,
    batch_size=16,
    epochs=30
)

predictions = model.predict(testingXSet) 

# apparently this just works
predicted_classes = (predictions > 0.5).astype(int)

c:\Users\mahd\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/30
1148/1148 ━━━━━━━━━━━━━━━━━━━━ 10s 6ms/step - AUC: 0.5024 - Precision: 0.4913 - Recall: 0.5822 - accuracy: 0.5023 - loss: 0.6933
Epoch 2/30
1148/1148 ━━━━━━━━━━━━━━━━━━━━ 7s 6ms/step - AUC: 0.5419 - Precision: 0.5427 - Recall: 0.4277 - accuracy: 0.5321 - loss: 0.6912
Epoch 3/30
1148/1148 ━━━━━━━━━━━━━━━━━━━━ 7s 6ms/step - AUC: 0.6182 - Precision: 0.5887 - Recall: 0.4708 - accuracy: 0.5736 - loss: 0.6704
Epoch 4/30
1148/1148 ━━━━━━━━━━━━━━━━━━━━ 7s 6ms/step - AUC: 0.6521 - Precision: 0.5967 - Recall: 0.5583 - accuracy: 0.5938 - loss: 0.6499
Epoch 5/30
1148/1148 ━━━━━━━━━━━━━━━━━━━━ 7s 6ms/step - AUC: 0.6746 - Precision: 0.6093 - Recall: 0.6170 - accuracy: 0.6110 - loss: 0.6348
Epoch 6/30
1148/1148 ━━━━━━━━━━━━━━━━━━━━ 7s 6ms/step - AUC: 0.6743 - Precision: 0.6004 - Recall: 0.6655 - accuracy: 0.6130 - loss: 0.6316
Epoch 7/30
1148/1148 ━━━━━━━━━━━━━━━━━━━━ 7s 6ms/step - AUC: 0.6799 - Precision: 0.5950 - Recall: 0.7171 - accuracy: 0.6160 - loss: 0.6246
Epoch 8/30
1148/1148 ━━━━━

In [7]:
confusionMatrix = confusion_matrix(testingYSet, predicted_classes)
print("Confusion Matrix:\n", confusionMatrix)

accuracy = accuracy_score(testingYSet, predicted_classes)
print("Accuracy:\n", accuracy)

recall = recall_score(testingYSet, predicted_classes)
print("Recall:\n", recall)

f1Score = f1_score(testingYSet, predicted_classes)
print("F1 Score:\n", f1Score)

Confusion Matrix:
 [[1163  438]
 [ 195 1443]]
Accuracy:
 0.8045693115158999
Recall:
 0.8809523809523809
F1 Score:
 0.8201193520886616


In [8]:
# its kinda weird, but it needs a dataset of representations to quantize to see what to scale immediate activation weights as
def representative_dataset():
    for i in range(400):  # take ~100 samples from your training set
        # x_train[i] shape: (96, 96, 1), values between 0 and 1
        # Must add batch dimension
        yield [trainingXSet[i:i+1].astype('float32')]


# begin converting, not only to tflite but also doing quantization

# specifies what we're gonna convert to
converter = tf.lite.TFLiteConverter.from_keras_model(model)

# give it the function to calculate middle activations
converter.representative_dataset = representative_dataset

converter.optimizations = [tf.lite.Optimize.DEFAULT]

# Force int8 quantization for both weights and activations
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]

# Set input/output to uint8
converter.inference_input_type = tf.int8
converter.inference_output_type = tf.int8

In [9]:
# actually make the quantized small model
finalModel = converter.convert()

# now, save it to a file
with open("../cnn_model_uint8.tflite", "wb") as f:
    f.write(finalModel)


INFO:tensorflow:Assets written to: C:\Users\mahd\AppData\Local\Temp\tmplkn3fqxi\assets


INFO:tensorflow:Assets written to: C:\Users\mahd\AppData\Local\Temp\tmplkn3fqxi\assets


Saved artifact at 'C:\Users\mahd\AppData\Local\Temp\tmplkn3fqxi'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 96, 96, 1), dtype=tf.float32, name='keras_tensor')
Output Type:
  TensorSpec(shape=(None, 1), dtype=tf.float32, name=None)
Captures:
  2177077873936: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2177077875088: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2177077873744: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2177077873360: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2177077873168: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2177077874512: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2177077872208: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2177077874704: TensorSpec(shape=(), dtype=tf.resource, name=None)


c:\Users\mahd\AppData\Local\Programs\Python\Python311\Lib\site-packages\tensorflow\lite\python\convert.py:854: UserWarning: Statistics for quantized inputs were expected, but not specified; continuing anyway.
  warnings.warn(
